In [15]:
import pandas as pd 

df = pd.read_csv('cosas/champions_23_24.csv')
df2 = pd.read_csv('cosas/champions.csv')
df3 = pd.read_csv('cosas/df_entrenador_trayectoria.csv')
df4 = pd.read_csv('cosas/df_entrenador.csv')
df5 = pd.read_csv('cosas/df_equipo.csv')
df6 = pd.read_csv('cosas/df_jugadores.csv')

df7 = pd.read_csv('dataframe/champions_23_24.csv')
df8 = pd.read_csv('dataframe/champions.csv')
df9 = pd.read_csv('dataframe/df_entrenador_trayectoria.csv')
df10 = pd.read_csv('dataframe/df_entrenador.csv')
df11 = pd.read_csv('dataframe/df_equipo.csv')
df12 = pd.read_csv('dataframe/df_jugadores.csv')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idPartido             124 non-null    int64  
 1   Temporada             124 non-null    object 
 2   Ronda                 124 non-null    object 
 3   Local                 124 non-null    int64  
 4   Visitante             124 non-null    int64  
 5   Evento                124 non-null    object 
 6   GolesLocal            124 non-null    int64  
 7   GolesVisitante        124 non-null    int64  
 8   VictoriaLocal         124 non-null    int64  
 9   Empate                124 non-null    int64  
 10  VictoriaVisitante     124 non-null    int64  
 11  %_Victorias_Local     124 non-null    float64
 12  %_Empate              124 non-null    float64
 13  %_Victoria_Visitante  124 non-null    float64
 14  %_Equipo1_Ganado      124 non-null    float64
 15  %_Equipo2_Ganado      1

In [16]:


df7.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idPartido             124 non-null    int64  
 1   Temporada             124 non-null    object 
 2   Ronda                 124 non-null    object 
 3   Local                 124 non-null    int64  
 4   Visitante             124 non-null    int64  
 5   Evento                124 non-null    object 
 6   GolesLocal            124 non-null    int64  
 7   GolesVisitante        124 non-null    int64  
 8   VictoriaLocal         124 non-null    int64  
 9   Empate                124 non-null    int64  
 10  VictoriaVisitante     124 non-null    int64  
 11  %_Victorias_Local     124 non-null    float64
 12  %_Empate              124 non-null    float64
 13  %_Victoria_Visitante  124 non-null    float64
 14  %_Equipo1_Ganado      124 non-null    float64
 15  %_Equipo2_Ganado      1

In [17]:
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idPartido             2494 non-null   int64  
 1   Temporada             2494 non-null   object 
 2   Ronda                 2494 non-null   object 
 3   Local                 2494 non-null   int64  
 4   Visitante             2494 non-null   int64  
 5   Evento                2494 non-null   object 
 6   GolesLocal            2494 non-null   int64  
 7   GolesVisitante        2494 non-null   int64  
 8   VictoriaLocal         2494 non-null   int64  
 9   Empate                2494 non-null   int64  
 10  VictoriaVisitante     2494 non-null   int64  
 11  %_Victorias_Local     2494 non-null   float64
 12  %_Empate              2494 non-null   float64
 13  %_Victoria_Visitante  2494 non-null   float64
 14  %_Equipo1_Ganado      2494 non-null   float64
 15  %_Equipo2_Ganado     

In [18]:
df8.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494 entries, 0 to 2493
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idPartido             2494 non-null   int64  
 1   Temporada             2494 non-null   object 
 2   Ronda                 2494 non-null   object 
 3   Local                 2494 non-null   int64  
 4   Visitante             2494 non-null   int64  
 5   Evento                2494 non-null   object 
 6   GolesLocal            2494 non-null   int64  
 7   GolesVisitante        2494 non-null   int64  
 8   VictoriaLocal         2494 non-null   int64  
 9   Empate                2494 non-null   int64  
 10  VictoriaVisitante     2494 non-null   int64  
 11  %_Victorias_Local     2494 non-null   float64
 12  %_Empate              2494 non-null   float64
 13  %_Victoria_Visitante  2494 non-null   float64
 14  %_Equipo1_Ganado      2494 non-null   float64
 15  %_Equipo2_Ganado     

In [19]:
df3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2119 entries, 0 to 2118
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idEntrenador  2119 non-null   int64 
 1   Temporada     2119 non-null   object
 2   Equipo        2119 non-null   int64 
 3   PJ            2119 non-null   int64 
 4   PG            2119 non-null   int64 
 5   PE            2119 non-null   int64 
 6   PP            2119 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 116.0+ KB


In [20]:
df9.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2119 entries, 0 to 2118
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idEntrenador  2119 non-null   int64 
 1   Temporada     2119 non-null   object
 2   Equipo        2119 non-null   int64 
 3   PJ            2119 non-null   int64 
 4   PG            2119 non-null   int64 
 5   PE            2119 non-null   int64 
 6   PP            2119 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 116.0+ KB


In [21]:
df4.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idEntrenador  250 non-null    int64 
 1   Apodo         250 non-null    object
 2   Nombre        250 non-null    object
 3   T             250 non-null    int64 
 4   PJ            250 non-null    int64 
 5   PG            250 non-null    int64 
 6   PE            250 non-null    int64 
 7   PP            250 non-null    int64 
 8   %             250 non-null    object
dtypes: int64(6), object(3)
memory usage: 17.7+ KB


In [22]:
df10.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idEntrenador  250 non-null    int64 
 1   Apodo         250 non-null    object
 2   Nombre        250 non-null    object
 3   T             250 non-null    int64 
 4   PJ            250 non-null    int64 
 5   PG            250 non-null    int64 
 6   PE            250 non-null    int64 
 7   PP            250 non-null    int64 
 8   %             250 non-null    object
dtypes: int64(6), object(3)
memory usage: 17.7+ KB


In [23]:
df5.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   idEquipo  553 non-null    int64 
 1   Nombre    553 non-null    object
 2   Pais      553 non-null    object
 3   T         553 non-null    int64 
 4   PJ        553 non-null    int64 
 5   PG        553 non-null    int64 
 6   PE        553 non-null    int64 
 7   PP        553 non-null    int64 
 8   %         553 non-null    object
 9   Títulos   553 non-null    int64 
 10  F         553 non-null    int64 
 11  1/2       553 non-null    int64 
 12  1/4       553 non-null    int64 
 13  1/8       553 non-null    int64 
 14  1/16      553 non-null    int64 
 15  GF        553 non-null    int64 
 16  GC        553 non-null    int64 
 17  TA        553 non-null    int64 
 18  TR        553 non-null    int64 
dtypes: int64(16), object(3)
memory usage: 82.2+ KB


In [24]:
df11.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553 entries, 0 to 552
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   idEquipo  553 non-null    int64 
 1   Nombre    553 non-null    object
 2   Pais      553 non-null    object
 3   T         553 non-null    int64 
 4   PJ        553 non-null    int64 
 5   PG        553 non-null    int64 
 6   PE        553 non-null    int64 
 7   PP        553 non-null    int64 
 8   %         553 non-null    object
 9   Títulos   553 non-null    int64 
 10  F         553 non-null    int64 
 11  1/2       553 non-null    int64 
 12  1/4       553 non-null    int64 
 13  1/8       553 non-null    int64 
 14  1/16      553 non-null    int64 
 15  GF        553 non-null    int64 
 16  GC        553 non-null    int64 
 17  TA        553 non-null    int64 
 18  TR        553 non-null    int64 
dtypes: int64(16), object(3)
memory usage: 82.2+ KB


In [25]:
df6.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20280 entries, 0 to 20279
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Temporada   20280 non-null  object 
 1   idJugador   20280 non-null  int64  
 2   Jugador     20280 non-null  object 
 3   Equipo      20280 non-null  int64  
 4   Edad        20280 non-null  float64
 5   PJ          20280 non-null  int64  
 6   Titular     20280 non-null  int64  
 7   Mín         20280 non-null  float64
 8   Gls.        20280 non-null  int64  
 9   Ass         20280 non-null  float64
 10  TP          20280 non-null  float64
 11  TPint       20280 non-null  float64
 12  TA          20280 non-null  int64  
 13  TR          20280 non-null  int64  
 14  xG          20280 non-null  float64
 15  npxG        20280 non-null  float64
 16  PrgC        20280 non-null  float64
 17  PrgP        20280 non-null  float64
 18  PrgR        20280 non-null  float64
 19  Valoracion  20280 non-nul

In [26]:
df12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20280 entries, 0 to 20279
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Temporada   20280 non-null  object 
 1   idJugador   20280 non-null  int64  
 2   Jugador     20280 non-null  object 
 3   País        20280 non-null  object 
 4   Posc        20280 non-null  object 
 5   Equipo      20280 non-null  int64  
 6   Edad        20280 non-null  float64
 7   PJ          20280 non-null  int64  
 8   Titular     20280 non-null  int64  
 9   Mín         20280 non-null  float64
 10  Gls.        20280 non-null  int64  
 11  Ass         20280 non-null  float64
 12  TP          20280 non-null  float64
 13  TPint       20280 non-null  float64
 14  TA          20280 non-null  int64  
 15  TR          20280 non-null  int64  
 16  xG          20280 non-null  float64
 17  npxG        20280 non-null  float64
 18  PrgC        20280 non-null  float64
 19  PrgP        20280 non-nul